In [42]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.svm import SVR
import numpy as np
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.base import TransformerMixin

In [31]:
hourly=pd.read_csv('./hour.csv')
daily=pd.read_csv('./hour.csv')

In [32]:
hourly.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [33]:
daily.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [34]:
X1_Train = hourly[hourly['dteday'] < '2012-10-01'].drop(['cnt','registered','dteday'],axis=1) ## CASUAL IN DATA
X2_Train =  hourly[hourly['dteday'] < '2012-10-01'].drop(['cnt','casual','dteday'],axis=1) ## REGISTERED IN DATA 
X3_Train = hourly[hourly['dteday'] < '2012-10-01'].drop(['cnt','casual','registered','dteday'],axis=1)## NONE OF THEM IN DATA

X1_Test = hourly[hourly['dteday'] >= '2012-10-01'].drop(['cnt','registered','dteday'],axis=1)## CASUAL IN DATA
X2_Test =  hourly[hourly['dteday'] >='2012-10-01'].drop(['cnt','casual','dteday'],axis=1)## REGISTERED IN DATA
X3_Test = hourly[hourly['dteday'] >= '2012-10-01'].drop(['cnt','casual','registered','dteday'],axis=1)## NONE OF THEM IN DATA

y_cas_train =hourly['casual'][hourly['dteday'] < '2012-10-01']
y_cnt_train =hourly['cnt'][hourly['dteday'] < '2012-10-01']
y_reg_train =hourly['registered'][hourly['dteday'] < '2012-10-01']

y_cas_test =hourly['casual'][hourly['dteday'] >= '2012-10-01']
y_cnt_test =hourly['cnt'][hourly['dteday'] >= '2012-10-01']
y_reg_test =hourly['registered'][hourly['dteday'] >= '2012-10-01']

In [111]:
# Create linear regression object
regr = LinearRegression(fit_intercept =True)
regr.fit(X2_Train, y_cas_train) # Using Registered & Using Casual
y_pred = regr.predict(X2_Test) # Using Registered for predicting
r2_score(y_cnt_test,y_reg_test + y_pred) # R2 score on Casual + Registered = Total 

0.968462574021778

In [36]:
#print(y_cnt_test)
#print(y_reg_test + y_pred)
#print(np.ravel(y_cnt_test),np.ravel((y_reg_test + y_pred)))

In [37]:
#SVR_Lin = SVR(kernel='linear',C=1e3)
#SVR_Lin.fit(X2_Train, y_cas_train)
#y_pred=SVR_Lin.predict(X2_Test)
#SVR_Lin.score(np.ravel(y_cnt_test),np.ravel((y_reg_test + y_pred)))

In [38]:
#regr_2 = AdaBoostRegressor(regr.fit(X2_Train, y_cas_train))
#regr_2.fit(X2_Train, y_cas_train)
#y_2 = regr_2.predict(X2_Test)


In [39]:
#regr_2.estimator_errors_ 

In [40]:
polynomial_features=PolynomialFeatures(degree=2)
x_poly=polynomial_features.fit_transform(X2_Train)
X_test_poly=polynomial_features.fit_transform(X2_Test)
model=LinearRegression()
model.fit(x_poly, y_cas_train)
y_poly_pred = model.predict(X_test_poly)
r2 = r2_score(y_cnt_test,y_reg_test + y_poly_pred)

In [41]:
r2

0.9828345358152757

In [130]:
dataset = pd.DataFrame({'y_poly_pred':y_poly_pred+y_reg_test,'Y_pred':y_pred+y_reg_test,'actual':actual})
dataset.head()

,y_poly_pred,Y_pred,actual
15211,39.491371,32.261023,45
15212,14.707740,-0.660619,18
15213,12.621222,-3.555586,12
15214,13.105325,-7.290061,7
15215,17.493575,-6.833103,10


In [131]:
stacker= LinearRegression()
stacker.fit(dataset[['y_poly_pred', 'Y_pred']], dataset['actual'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [132]:
temp=dataset[['y_poly_pred','Y_pred']]

In [133]:
tar=dataset[['actual']]

In [134]:
prediction=stacker.predict(temp)

In [135]:
r2_score(tar,prediction)

0.9869560808369517